![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

**Note:** Notebook code generated using Prompt Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Prompt Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.10.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [3]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : "G7HLLKrhe-Zl2sJcYH9yF3MFs5E14Sj9zJhho_bvWE9B"
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [4]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [5]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 500,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [6]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


In [7]:
project_id

'404e8279-d931-4311-b13d-4327914f2b8c'

In [8]:
space_id

## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [9]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id
	#space_id = space_id
	)


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [6]:
prompt_input = """<<SYS>>
{"الدور": "النظام"، "المحتوى": "أنت مساعد ذكاء صنعي خبير في اللغة العربية"}
{"الدور": "المستخدم"، "المحتوى": "إنشاء 10 كلمات (كلمات عربية). إنشاء كلمة ونوع قسم الكلام (فعل، اسم، حرف)، يتم إخراجها بدقة بتنسيق JSON كقائمة قواميس ضمن متغير اسمه generated_words. 
يجب أن يحتوي كل قاموس على مفتاحين 'Word' و'Type' "}
<</SYS>>"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [7]:
question = input("Question: ")
formattedQuestion = f"""<s> [INST] {question} [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")


Question: 
AI:  [
  {
    "w": "مشرق",
    "t": "اسم"
  },
  {
    "w": "بدأ",
    "t": "فعل"
  },
  {
    "w": "لكن",
    "t": "حرف"
  }
] 

هذه ثلاثة كلمات عربية مع تصنيفها النحوي: مشرق (اسم)، بدأ (فعل)، لكن (حرف). تم تنسيق الإجابة على شكل قائمة قواميس JSON كما طلبت. 


In [7]:
formattedQuestion = f"""<s> [INST] ابدأ [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

AI:  بالطبع يمكنني مساعدتك في إنشاء قائمة تضم 10 كلمات عربية مع تصنيفها إلى أنواع الكلام (أفعال، أسماء، حروف). سأقدم لك هذه المعلومات بتنسيق JSON ضمن متغير يسمى `generated_words`. كل قاموس في القائمة سيحتوي على مفتاحين: 'Word' و'Type'.

إليك قائمة الكلمات المطلوبة:

```json
{
  "generated_words": [
    {
      "Word": "ابتكار",
      "Type": "اسم"
    },
    {
      "Word": "تعلم",
      "Type": "فعل"
    },
    {
      "Word": "من",
      "Type": "حرف"
    },
    {
      "Word": "مساعدة",
      "Type": "اسم"
    },
    {
      "Word": "أعطى",
      "Type": "فعل"
    },
    {
      "Word": "إلى",
      "Type": "حرف"
    },
    {
      "Word": "مجتمع",
      "Type": "اسم"
    },
    {
      "Word": "ساعد",
      "Type": "فعل"
    },
    {
      "Word": "على",
      "Type": "حرف"
    },
    {
      "Word": "تطوير",
      "Type": "اسم"
    }
  ]
}
```

يرجى ملاحظة أن هذه الكلمات تم اختيارها بشكل عشوائي وقد لا تتوافق مع المعايير المحددة بدقة 100%. إذا كنت بحاجة إلى مزيد من المساعدة أو لديك

In [8]:
!pip install regex
import regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 27.5 MB/s eta 0:00:00


In [9]:
pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
words = pattern.findall(generated_response)

In [10]:
import json
parsed_words = json.loads(words[0])
print(parsed_words)

{'generated_words': [{'Word': 'ابتكار', 'Type': 'اسم'}, {'Word': 'تعلم', 'Type': 'فعل'}, {'Word': 'من', 'Type': 'حرف'}, {'Word': 'مساعدة', 'Type': 'اسم'}, {'Word': 'أعطى', 'Type': 'فعل'}, {'Word': 'إلى', 'Type': 'حرف'}, {'Word': 'مجتمع', 'Type': 'اسم'}, {'Word': 'ساعد', 'Type': 'فعل'}, {'Word': 'على', 'Type': 'حرف'}, {'Word': 'تطوير', 'Type': 'اسم'}]}


In [11]:
parsed_words['generated_words']

[{'Word': 'ابتكار', 'Type': 'اسم'},
 {'Word': 'تعلم', 'Type': 'فعل'},
 {'Word': 'من', 'Type': 'حرف'},
 {'Word': 'مساعدة', 'Type': 'اسم'},
 {'Word': 'أعطى', 'Type': 'فعل'},
 {'Word': 'إلى', 'Type': 'حرف'},
 {'Word': 'مجتمع', 'Type': 'اسم'},
 {'Word': 'ساعد', 'Type': 'فعل'},
 {'Word': 'على', 'Type': 'حرف'},
 {'Word': 'تطوير', 'Type': 'اسم'}]

In [40]:
len(parsed_words['generated_words'])

10

In [63]:
# Creating separate lists for words and types
words = [item['Word'] for item in parsed_words['generated_words']]
types = [item['Type'] for item in parsed_words['generated_words']]

#print("\nWords:", words)
#print("Types:", types)

# Filtering words based on their type
#nouns = [item['Word'] for item in word_data if item['Type'] == 'اسم']
#print("\nNouns:", nouns)

In [20]:
parsed_words_with_answers = parsed_words

In [23]:
parsed_words_with_answers

{'generated_words': [{'Word': 'ابتكار', 'Type': 'اسم'},
  {'Word': 'تعلم', 'Type': 'فعل'},
  {'Word': 'من', 'Type': 'حرف'},
  {'Word': 'مساعدة', 'Type': 'اسم'},
  {'Word': 'أعطى', 'Type': 'فعل'},
  {'Word': 'إلى', 'Type': 'حرف'},
  {'Word': 'مجتمع', 'Type': 'اسم'},
  {'Word': 'ساعد', 'Type': 'فعل'},
  {'Word': 'على', 'Type': 'حرف'},
  {'Word': 'تطوير', 'Type': 'اسم'}]}

In [24]:
# Accessing individual words and their types
user_answer = []
i = 0
for item in parsed_words['generated_words']:
    
    word = item['Word']
    word_type = item['Type']
    #print(f"Word: {word}, Type: {word_type}")
    print(' ما نوع كلمة: ' + word)
    parsed_words_with_answers['generated_words'][i]['answer'] = input("الجواب: ")
    i+=1
    print('\n')

 ما نوع كلمة: ابتكار
الجواب: اسم


 ما نوع كلمة: تعلم
الجواب: فعل


 ما نوع كلمة: من
الجواب: اسم


 ما نوع كلمة: مساعدة
الجواب: اسم


 ما نوع كلمة: أعطى
الجواب: فعل


 ما نوع كلمة: إلى
الجواب: اسم


 ما نوع كلمة: مجتمع
الجواب: فعل


 ما نوع كلمة: ساعد
الجواب: فعل


 ما نوع كلمة: على
الجواب: حرف


 ما نوع كلمة: تطوير
الجواب: اسم




In [25]:
parsed_words_with_answers

{'generated_words': [{'Word': 'ابتكار', 'Type': 'اسم', 'answer': 'اسم'},
  {'Word': 'تعلم', 'Type': 'فعل', 'answer': 'فعل'},
  {'Word': 'من', 'Type': 'حرف', 'answer': 'اسم'},
  {'Word': 'مساعدة', 'Type': 'اسم', 'answer': 'اسم'},
  {'Word': 'أعطى', 'Type': 'فعل', 'answer': 'فعل'},
  {'Word': 'إلى', 'Type': 'حرف', 'answer': 'اسم'},
  {'Word': 'مجتمع', 'Type': 'اسم', 'answer': 'فعل'},
  {'Word': 'ساعد', 'Type': 'فعل', 'answer': 'فعل'},
  {'Word': 'على', 'Type': 'حرف', 'answer': 'حرف'},
  {'Word': 'تطوير', 'Type': 'اسم', 'answer': 'اسم'}]}

In [26]:
model_id = "sdaia/allam-1-13b-instruct"
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 200,
    "repetition_penalty": 1
}
model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

Please enter your api key (hit enter): ········


In [ ]:
Input: {{'Word': {parsed_words_with_answers[['generated_words']][[i]][['word']]}, 'Type': {parsed_words_with_answers[['generated_words']][[i]][['answer']]}}}

In [31]:
parsed_words_with_answers['generated_words'][1]['Word']

'تعلم'

In [35]:
input1 = parsed_words_with_answers['generated_words'][2]['Word']
type1 =  parsed_words_with_answers['generated_words'][2]['answer']


prompt_input = f"""أنت نموذج لكشف صحة قسم الكلام. صنف السؤال المدخل لصحيح أو خاطئ

Input: {{'Word': 'أفكار', 'Type': 'اسم'}}
Output: {{'صح','الجواب'}}

Input: {{'Word': 'كتب', 'Type': 'فعل'}}
Output: {{'صح','الجواب'}}

Input: {{'Word': 'على', 'Type': 'حرف'}}
Output: {{'صح','الجواب'}}

Input: {{'Word': 'يقرأ', 'Type': 'حرف'}}
Output: {{'خطأ','الجواب'}}

Input: {{'Word': {input1}, 'Type': {type1}}}
Output:"""

In [36]:
print("Submitting generation request...")
generated_response = model.generate_text(prompt=prompt_input, guardrails=False)
print(input1,type1,'\n')
print(generated_response)

Submitting generation request...
من اسم 

 {'خطأ','الجواب'} 


In [34]:
print(input1,type1)

تعلم فعل


In [ ]:
import requests

url = "https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"

body = {
	"input": """أنت نموذج لكشف صحة قسم الكلام. صنف السؤال المدخل لصحيح أو خاطئ

Input: {'\''Word'\'': '\''أفكار'\'', '\''Type'\'': '\''اسم'\''}
Output: {'\''صح'\'','\''الجواب'\''}

Input: {'\''Word'\'': '\''كتب'\'', '\''Type'\'': '\''فعل'\''}
Output: {'\''صح'\'','\''الجواب'\''}

Input: {'\''Word'\'': '\''على'\'', '\''Type'\'': '\''حرف'\''}
Output: {'\''صح'\'','\''الجواب'\''}

Input: {'\''Word'\'': '\''يقرأ'\'', '\''Type'\'': '\''حرف'\''}
Output: {'\''خطأ'\'','\''الجواب'\''}

Input: {'\''Word'\'': شمس, '\''Type'\'': حرف}
Output:""",
	"parameters": {
		"decoding_method": "greedy",
		"max_new_tokens": 200,
		"repetition_penalty": 1
	},
	"model_id": "sdaia/allam-1-13b-instruct",
	"project_id": "404e8279-d931-4311-b13d-4327914f2b8c"
}

headers = {
	"Accept": "application/json",
	"Content-Type": "application/json",
	"Authorization": "Bearer YOUR_ACCESS_TOKEN"
}

response = requests.post(
	url,
	headers=headers,
	json=body
)

if response.status_code != 200:
	raise Exception("Non-200 response: " + str(response.text))

data = response.json()

# With tashkeel

In [14]:
prompt_input = """<<SYS>>
{"الدور": "النظام"، "المحتوى": "أنت مساعد ذكاء صنعي خبير في اللغة العربية"}
{"الدور": "المستخدم"، "المحتوى": "إنشاء 10 كلمات (كلمات عربية مشكلة مثل: كَتَبَ، مُستودَع). إنشاء كلمة ونوع قسم الكلام (فعل، اسم، حرف)، يتم إخراجها بدقة بتنسيق JSON كقائمة قواميس ضمن متغير اسمه generated_words. 
يجب أن يحتوي كل قاموس على مفتاحين 'Word' و'Type' 
تجنب تكرار الكلمات"}
<</SYS>>"""


In [15]:
formattedQuestion = f"""<s> [INST] ابدأ [/INST]"""
prompt = f"""{prompt_input}{formattedQuestion}"""
generated_response = model.generate_text(prompt=prompt, guardrails=False)
print(f"AI: {generated_response}")

AI:  بالطبع! سأقوم بإنشاء 10 كلمات عربية مشكلة مع تحديد نوع الكلمة (فعل أو اسم أو حرف) لكل كلمة. سأقدم المعلومات بتنسيق JSON كما طلبت:

فيما يلي قائمة بالكلمات المولدة مع أنواعها:

[
  {
    "Word": "قَرَأ",
    "Type": "فعل"
  },
  {
    "Word": "مُستودَع",
    "Type": "اسم"
  },
  {
    "Word": "عَلِم",
    "Type": "فعل"
  },
  {
    "Word": "مَدْرَسَة",
    "Type": "اسم"
  },
  {
    "Word": "أَخَذ",
    "Type": "فعل"
  },
  {
    "Word": "مُحَاضَرَة",
    "Type": "اسم"
  },
  {
    "Word": "أَنجَح",
    "Type": "فعل"
  },
  {
    "Word": "مُؤَسَّسَة",
    "Type": "اسم"
  },
  {
    "Word": "أَعَانَ",
    "Type": "فعل"
  },
  {
    "Word": "مُؤَلِّف",
    "Type": "اسم"
  }
]

يرجى ملاحظة أن هذه الكلمات تم إنشاؤها بناءً على قواعد اللغة العربية وقد تكون مكررة بشكل طفيف. إذا كنت بحاجة إلى المزيد من الكلمات أو المساعدة، لا تتردد في السؤال. 
